In [1]:
import os, glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch

from ultralytics import YOLO
import easyocr
import utils

In [2]:
if "temp" not in os.listdir():
    os.makedirs("./temp")

<h3 style="color:orange">Hyperparameters</h3>

In [3]:
DEVICE="cuda" if torch.cuda.is_available() else "cpu"

print(f"{DEVICE=}")

vid_dir=["./dataset/video/demo.mp4"]

vid=cv2.VideoCapture(vid_dir[0])

# video metadata
frame_count=int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
W=int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
H=int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
FPS=int(vid.get(cv2.CAP_PROP_FPS))

print(f"frame count: {frame_count}\nframe width: {W}\nframe hight: {H}\nFPS: {FPS}")

# save video folder
# out_mp4=cv2.VideoWriter("./temp/video/ans.mp4",cv2.VideoWriter_fourcc(*"mp4v"),FPS,(W,H))

DEVICE='cuda'
frame count: 631
frame width: 1114
frame hight: 720
FPS: 30


<h3 style="color:orange">Load Model</h3>

In [4]:
model_weight=[path for path in glob.glob("./Model/weights/*.pt")]
print(model_weight)
model=YOLO(model_weight[1]).to(DEVICE)

matter_tracker=utils.sort.Sort()   #use for tracting object in continues frames

print(model)

['./Model/weights\\yolov8n.pt', './Model/weights\\YOLOv8_LPD_largeset.pt', './Model/weights\\YOLOv8_LPD_roboflow.pt']
YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1,

<h3 style="color:orange">model infernce</h3>

In [ ]:
results={}

if "video" not in os.listdir("./temp/"): os.makedirs("./temp/video")
out_mp4=cv2.VideoWriter("./temp/video/ans.mp4",cv2.VideoWriter_fourcc(*"mp4v"),FPS,(W,H))

LIMIT=50

ret,frame_num=True,-1
while(ret):
    frame_num+=1
    ret,frame=vid.read()
    print(f"{ret}, Frame no. {frame_num}")
    if(ret):

        # if(frame_num>LIMIT): break     #debug
        
        results[frame_num]={}

        # detect numberplate
        detections=model(frame)[0]
        detection_list=[]
        for detection in detections.boxes.data.tolist():
            x1,y1,x2,y2,score,class_id=detection
            if(score>=0.5):
                # print([int(x1),int(y1),int(x2),int(y2),score])
                detection_list.append([int(x1),int(y1),int(x2),int(y2),score])
        print("\n")
    

        ## track vehicle 
        # track_id=matter_tracker.update(np.asarray(detection_list))
        # print(track_id)

        # crop Lincence plate
        crop_license_plate=frame[int(y1):int(y2),int(x1):int(x2),:]
        # print(crop_license_plate.shape)
        if "crop" not in os.listdir("./temp/"): os.makedirs("./temp/crop")
        cv2.imwrite(f"./temp/crop/{frame_num}.png",crop_license_plate)    ## DEBUG

        ## correct one
        ans=utils.utils.read_license_plate(crop_license_plate)
        # print(ans,len(ans))

        if(len(ans)==0): continue
        # wrting test sample in ans_img
        if "ans_img" not in os.listdir("./temp/"): os.makedirs("./temp/ans_img")

        # add color base on correct OCR identification
        color=(0,255,0)
        for text,score,Flag in ans:
            print(Flag)
            if Flag: color=(0,255,0)        #BGR
            else: color=(0,0,255)           #BGR

            cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),(int(y2))),color,3)
            cv2.imwrite(f"./temp/ans_img/{frame_num}.png",frame)    ## DEBUG

            
        
        ##  save video
        out_mp4.write(frame)

# clear all video caps
out_mp4.release()
cv2.destroyAllWindows()

True, Frame no. 0

0: 416x640 2 License Platess, 40.1ms
Speed: 3.1ms preprocess, 40.1ms inference, 74.4ms postprocess per image at shape (1, 3, 416, 640)


False
True, Frame no. 1

0: 416x640 2 License Platess, 6.6ms
Speed: 1.5ms preprocess, 6.6ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 640)


False
True, Frame no. 2

0: 416x640 2 License Platess, 6.7ms
Speed: 1.4ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)


False
True, Frame no. 3

0: 416x640 3 License Platess, 6.0ms
Speed: 1.4ms preprocess, 6.0ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)


True, Frame no. 4

0: 416x640 3 License Platess, 5.9ms
Speed: 1.7ms preprocess, 5.9ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)


True, Frame no. 5

0: 416x640 3 License Platess, 6.3ms
Speed: 1.4ms preprocess, 6.3ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)


True, Frame no. 6

0: 416x640 3 License Platess, 6.4ms
Speed: